In [1]:
import wandb
import pandas as pd

# fetch the artifact sample.csv from W&B and read it with panda
# we use save_code=True in the call to wandb.init so the notebook is uploaded 
# and versioned by W&B.
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: geodego (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


### Profiling

 We use pandas-profiling to perform eda on raw datas

In [2]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### EDA
- There are missing values in 2 columns last_review and reviews_per_month.
- The variable price has some outliers: we decide to consider from a minimum of USD10 to a maximum of USD350 per night.
- the column last_review is a date but it is in string format.



### Problems Fix

In [3]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

We did not impute missing values. We will do that in the inference pipeline, so we will be able to handle missing values also in production.

### Check if problems are fixed

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

In [5]:
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

- last_review is now of type datetime
- the column price has no longer outliers

In [6]:
# terminate the run
run.finish()

In [7]:
# to Save the notebook, then close it (File -> Close and Halt). 
# In the main Jupyter notebook page, click Quit in the upper right to stop Jupyter. 
# This will also terminate the mlflow run. DO NOT USE CRTL-C